# vTTS - Universal TTS/STT Serving System 완전 테스트

**vLLM for Speech** - API 기반 TTS/STT 서빙 시스템

## 🎯 vTTS의 핵심: vLLM처럼 빠른 API 접근

```bash
# 1. 서버 시작
vtts serve Supertone/supertonic-2

# 2. API 호출 (즉시 사용 가능!)
curl http://localhost:8000/v1/audio/speech \
  -H "Content-Type: application/json" \
  -d '{"input": "Hello", "voice": "F1"}'
```

이 노트북에서는:
- ✅ **Part 0**: API 서버 테스트 (vLLM 스타일)
- ✅ **Part 1**: Supertonic-2 TTS (8 voices × 5 languages)
- ✅ **Part 2**: Faster-Whisper STT (모델 크기, 포맷)
- ✅ **Part 3**: TTS + STT Round-trip

## 🚀 Part 0: API 서버 테스트 (핵심 기능!)

vTTS의 가장 큰 장점: **vLLM처럼 간단한 API 접근**

In [ ]:
# vTTS + Supertonic 설치 (정확한 버전 고정!)
print("vTTS + Supertonic 설치 중...\n")

# 1. numpy + onnxruntime-gpu 버전 고정
print("  [1/2] numpy + onnxruntime-gpu 설치 (정확한 버전)...")
!pip uninstall numpy onnxruntime onnxruntime-gpu -y -q
!pip install -q "numpy==1.26.4" "onnxruntime-gpu==1.18.0"

# 2. vTTS 설치
print("  [2/2] vTTS 설치...")
!pip install -q "vtts[supertonic] @ git+https://github.com/bellkjtt/vTTS.git"

print("\n✅ vTTS 설치 완료!")

In [ ]:
# GPU 확인
import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    DEVICE = "cuda"
else:
    print("GPU 없음, CPU 모드로 실행")
    DEVICE = "cpu"

### 0.1 vTTS 서버 시작 (백그라운드)

In [ ]:
import subprocess
import time
import os

# 로그 파일 설정
log_file = open('vtts_server.log', 'w')

# 백그라운드로 서버 시작
print("🚀 Starting vTTS server in background...")
print("   Model: Supertone/supertonic-2")
print("   Port: 8000")
print("   STT: large-v3")

server_process = subprocess.Popen(
    ['vtts', 'serve', 'Supertone/supertonic-2', 
     '--host', '0.0.0.0',
     '--port', '8000',
     '--stt-model', 'base',  # Kaggle에서는 작은 모델
     '--log-level', 'INFO'],
    stdout=log_file,
    stderr=subprocess.STDOUT
)

print(f"   PID: {server_process.pid}")
print("\n⏳ Waiting for server to start...")

# 서버가 시작될 때까지 대기
import httpx

max_retries = 30
for i in range(max_retries):
    try:
        response = httpx.get('http://localhost:8000/health', timeout=2.0)
        if response.status_code == 200:
            print(f"✅ Server is ready! (took {i+1}s)")
            break
    except:
        if i < max_retries - 1:
            print(f"   Waiting... ({i+1}/{max_retries})")
            time.sleep(1)
        else:
            print("❌ Server failed to start. Check vtts_server.log")
            raise

print("\n📝 Server endpoints:")
print("   - TTS: http://localhost:8000/v1/audio/speech")
print("   - STT: http://localhost:8000/v1/audio/transcriptions")
print("   - Docs: http://localhost:8000/docs")
print("   - Models: http://localhost:8000/v1/models")

### 🔍 디버깅: 서버 로그 확인 (에러 발생 시)

In [ ]:
# 에러 발생 시 서버 로그 확인
print("📋 최근 서버 로그 (마지막 50줄):\n" + "="*60)
!tail -n 50 vtts_server.log

In [ ]:
# 서버 재시작이 필요한 경우
print("🔄 서버 재시작...\n")

# 기존 프로세스 종료
try:
    server_process.terminate()
    server_process.wait(timeout=5)
    print("✅ 기존 서버 종료 완료")
except:
    print("⚠️ 서버가 실행중이 아니거나 이미 종료됨")

# 새로운 서버 시작
import subprocess
log_file = open('vtts_server.log', 'w')
server_process = subprocess.Popen(
    ['vtts', 'serve', 'Supertone/supertonic-2', 
     '--host', '0.0.0.0',
     '--port', '8000',
     '--stt-model', 'base'],
    stdout=log_file,
    stderr=subprocess.STDOUT
)

import time
import httpx
time.sleep(10)  # 서버 시작 대기

try:
    response = httpx.get('http://localhost:8000/health', timeout=5.0)
    print(f"✅ 서버 재시작 완료! Status: {response.status_code}")
except Exception as e:
    print(f"❌ 서버 시작 실패: {e}")
    print("\n📋 로그 확인:")
    !tail -n 30 vtts_server.log

### 0.2 API 테스트 #1: vTTS 클라이언트 (가장 간단)

In [ ]:
from vtts import VTTSClient
import IPython.display as ipd

# 클라이언트 생성 (한 줄!)
client = VTTSClient(base_url="http://localhost:8000")

print("🎙️ TTS API 테스트\n" + "="*60)

# 음성 생성 (단 한 줄!)
audio = client.tts(
    text="안녕하세요, vTTS API 테스트입니다.",
    voice="F1",
    language="ko"
)

print(f"✅ 음성 생성 완료!")
print(f"   Duration: {len(audio.audio)/audio.sample_rate:.2f}s")
print(f"   Sample rate: {audio.sample_rate}Hz")

# 재생
display(ipd.Audio(audio.audio, rate=audio.sample_rate))

# 저장
audio.save("api_test_output.wav")
print("   Saved: api_test_output.wav")

### 0.3 API 테스트 #2: OpenAI SDK (호환성)

In [ ]:
from openai import OpenAI

# OpenAI SDK로 vTTS 사용 (완전 호환!)
openai_client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="dummy"  # vTTS는 API key 불필요
)

print("🔊 OpenAI API 호환성 테스트\n" + "="*60)

# TTS (OpenAI 스타일)
response = openai_client.audio.speech.create(
    model="Supertone/supertonic-2",
    voice="M1",
    input="Hello, this is OpenAI compatible API test."
)

# 저장
response.stream_to_file("openai_compatible.mp3")
print("✅ OpenAI API 호환 확인!")
print("   Saved: openai_compatible.mp3")

# 재생
display(ipd.Audio("openai_compatible.mp3"))

### 0.4 API 테스트 #3: HTTP 직접 호출 (curl 스타일)

In [ ]:
import httpx
import json

print("🌐 HTTP API 직접 호출 테스트\n" + "="*60)

# TTS API 호출
tts_payload = {
    "model": "Supertone/supertonic-2",
    "input": "This is HTTP API test.",
    "voice": "F2",
    "language": "en",
    "response_format": "wav"
}

response = httpx.post(
    "http://localhost:8000/v1/audio/speech",
    json=tts_payload,
    timeout=30.0
)

if response.status_code == 200:
    with open("http_api_test.wav", "wb") as f:
        f.write(response.content)
    print("✅ HTTP API 호출 성공!")
    print(f"   Status: {response.status_code}")
    print(f"   Content-Type: {response.headers.get('content-type')}")
    print(f"   Size: {len(response.content)} bytes")
    print("   Saved: http_api_test.wav")
    
    display(ipd.Audio("http_api_test.wav"))
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

### 0.5 API 테스트 #4: STT (Whisper API 호환)

In [ ]:
print("🎤 STT API 테스트 (OpenAI Whisper 호환)\n" + "="*60)

# TTS로 생성한 오디오를 STT로 인식
with open("api_test_output.wav", "rb") as f:
    transcription = openai_client.audio.transcriptions.create(
        model="base",
        file=f,
        language="ko"
    )

print("✅ STT API 호출 성공!")
print(f"   원본: 안녕하세요, vTTS API 테스트입니다.")
print(f"   인식: {transcription.text}")

### 0.6 API 테스트 #5: 모델 목록 조회

In [ ]:
print("📋 모델 목록 조회\n" + "="*60)

# /v1/models 엔드포인트
models_response = httpx.get("http://localhost:8000/v1/models")
models_data = models_response.json()

print("✅ 사용 가능한 모델:")
for model in models_data['data']:
    print(f"   - {model['id']}")
    print(f"     Owner: {model.get('owned_by', 'N/A')}")
    print()

### 0.7 API 성능 벤치마크

In [ ]:
import time

print("⏱️ API 성능 벤치마크\n" + "="*60)

# 10회 반복 테스트
num_tests = 10
times = []

test_text = "Performance test."

for i in range(num_tests):
    start = time.time()
    
    response = httpx.post(
        "http://localhost:8000/v1/audio/speech",
        json={
            "model": "Supertone/supertonic-2",
            "input": test_text,
            "voice": "F1",
            "response_format": "wav"
        },
        timeout=30.0
    )
    
    elapsed = time.time() - start
    times.append(elapsed)
    
    if i == 0:
        print(f"Run {i+1}: {elapsed:.3f}s (first run - model loading)")
    else:
        print(f"Run {i+1}: {elapsed:.3f}s")

avg_time = sum(times[1:]) / len(times[1:])  # 첫 실행 제외

print(f"\n📊 벤치마크 결과:")
print(f"   첫 실행 (모델 로딩): {times[0]:.3f}s")
print(f"   평균 응답 시간: {avg_time:.3f}s")
print(f"   최소: {min(times[1:]):.3f}s")
print(f"   최대: {max(times[1:]):.3f}s")
print(f"\n✅ vLLM 스타일의 빠른 API 서빙 확인!")

## 📊 Part 0 요약

### ✅ vTTS API 테스트 완료!

**테스트한 방식:**
1. ✅ vTTS 클라이언트 (Python SDK)
2. ✅ OpenAI SDK (완전 호환)
3. ✅ HTTP 직접 호출 (curl 스타일)
4. ✅ STT API (Whisper 호환)
5. ✅ 모델 목록 조회
6. ✅ 성능 벤치마크

**핵심 장점:**
- 🚀 vLLM처럼 간단한 API 접근
- 🔌 OpenAI API 완전 호환
- ⚡ 빠른 응답 속도
- 🌐 RESTful API

---

이제부터는 각 엔진의 세부 파라미터 테스트입니다.

## 🎙️ Part 1: Supertonic-2 TTS 전체 테스트

### 테스트 파라미터:
- **음성 스타일**: M1, M2, M3, M4, F1, F2, F3, F4 (8개)
- **언어**: en, ko, es, pt, fr (5개)
- **총 조합**: 40개 테스트 케이스

In [ ]:
# API를 통해 모든 음성 스타일 테스트
import soundfile as sf

test_texts = {
    "ko": "안녕하세요, vTTS 테스트입니다.",
    "en": "Hello, this is vTTS testing.",
    "es": "Hola, esta es una prueba de vTTS.",
    "pt": "Olá, este é um teste vTTS.",
    "fr": "Bonjour, ceci est un test vTTS."
}

all_voices = ["M1", "M2", "M3", "M4", "F1", "F2", "F3", "F4"]

print("🎙️ API를 통한 음성 스타일 테스트\n" + "="*60)

os.makedirs("outputs/api_tests", exist_ok=True)

for voice in all_voices[:3]:  # 처음 3개만 (시간 절약)
    print(f"\n🔊 Voice: {voice}")
    
    audio = client.tts(
        text=test_texts["ko"],
        voice=voice,
        language="ko"
    )
    
    filename = f"outputs/api_tests/{voice}.wav"
    audio.save(filename)
    
    print(f"   Duration: {len(audio.audio)/audio.sample_rate:.2f}s")
    print(f"   File: {filename}")
    display(ipd.Audio(audio.audio, rate=audio.sample_rate))

print("\n✅ API 음성 스타일 테스트 완료!")

## 🎤 Part 2: Faster-Whisper STT API 테스트

### 다양한 출력 포맷 테스트

In [ ]:
print("📄 STT 출력 포맷 테스트\n" + "="*60)

output_formats = ["json", "text", "srt", "vtt"]

for fmt in output_formats:
    print(f"\n📋 Format: {fmt}")
    
    # HTTP로 직접 호출
    with open("api_test_output.wav", "rb") as f:
        files = {'file': ('audio.wav', f, 'audio/wav')}
        data = {
            'model': 'base',
            'language': 'ko',
            'response_format': fmt
        }
        
        response = httpx.post(
            "http://localhost:8000/v1/audio/transcriptions",
            files=files,
            data=data,
            timeout=30.0
        )
    
    if response.status_code == 200:
        if fmt == "json":
            result = response.json()
            print(f"   Text: {result['text']}")
        else:
            result_text = response.text[:200]
            print(f"   Result: {result_text}...")
    else:
        print(f"   Error: {response.status_code}")

print("\n✅ STT API 포맷 테스트 완료!")

## 🔄 Part 3: API Round-trip 테스트

TTS → STT 전체 파이프라인을 API로 테스트

In [ ]:
print("🔄 API Round-trip 테스트\n" + "="*60)

roundtrip_results = []

for lang, original_text in list(test_texts.items())[:3]:  # 3개 언어만
    print(f"\n🔊 Language: {lang}")
    
    # 1. TTS로 음성 생성 (API)
    tts_audio = client.tts(
        text=original_text,
        voice="F1",
        language=lang
    )
    
    temp_file = f"temp_{lang}.wav"
    tts_audio.save(temp_file)
    
    # 2. STT로 인식 (API)
    with open(temp_file, "rb") as f:
        transcription = openai_client.audio.transcriptions.create(
            model="base",
            file=f,
            language=lang
        )
    
    print(f"   원본: {original_text}")
    print(f"   인식: {transcription.text}")
    
    roundtrip_results.append({
        "language": lang,
        "original": original_text,
        "transcribed": transcription.text
    })

print("\n✅ API Round-trip 테스트 완료!")

## 🎉 최종 요약

### vTTS API 테스트 결과

In [ ]:
print("\n" + "="*80)
print("🎉 vTTS API 테스트 완료!")
print("="*80)
print()
print("✅ 완료된 API 테스트:")
print("  🔌 vTTS Python Client")
print("  🔌 OpenAI SDK (완전 호환)")
print("  🔌 HTTP API (curl 스타일)")
print("  🎙️ TTS API (다양한 음성)")
print("  🎤 STT API (다양한 포맷)")
print("  🔄 Round-trip (TTS → STT)")
print("  📊 성능 벤치마크")
print()
print("🚀 vLLM 스타일 API 서빙 확인:")
print(f"  - 평균 응답 시간: {avg_time:.3f}s")
print(f"  - OpenAI API 호환: ✅")
print(f"  - RESTful API: ✅")
print()
print("📦 vTTS 사용법:")
print("  1. pip install 'vtts[all] @ git+https://github.com/bellkjtt/vTTS.git'")
print("  2. vtts serve Supertone/supertonic-2")
print("  3. curl http://localhost:8000/v1/audio/speech -d '{...}'")
print()
print("💡 핵심 장점:")
print("  - 🚀 vLLM처럼 간단한 API 접근")
print("  - 🔌 OpenAI API 완전 호환")
print("  - ⚡ 빠른 응답 속도")
print("  - 🌐 RESTful API")
print()
print("🌟 GitHub: https://github.com/bellkjtt/vTTS")
print()

## 🧹 서버 정리

In [ ]:
# 서버 종료
print("🛑 Stopping vTTS server...")
server_process.terminate()
server_process.wait(timeout=5)
log_file.close()
print("✅ Server stopped.")